# 1. Publication data cleaning

In [5]:
import pandas as pd

# Load the dataset
file_path = 'Data_clean/ViCom_Publications_Full.csv'
df = pd.read_csv(file_path, dtype={'Year': 'Int64'})  # Keep 'Year' as nullable integer

# Inspect the data
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711 entries, 0 to 1710
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Researcher                1711 non-null   object
 1   Year                      1707 non-null   Int64 
 2   Type                      1711 non-null   object
 3   Title                     1707 non-null   object
 4   Abstract                  1707 non-null   object
 5   Source                    1707 non-null   object
 6   Authors                   1707 non-null   object
 7   Database Affiliation      1711 non-null   object
 8   Publication Institutions  1365 non-null   object
 9   Matched Affiliation       1711 non-null   object
 10  Topic                     1707 non-null   object
 11  Review Flag               1711 non-null   object
dtypes: Int64(1), object(11)
memory usage: 162.2+ KB
None


In [6]:
# Handle missing values column-wise
df['Title'] = df['Title'].fillna('Unknown')
df['Abstract'] = df['Abstract'].fillna('Unknown')
df['Source'] = df['Source'].fillna('Unknown')
df['Authors'] = df['Authors'].fillna('Unknown')
df['Database Affiliation'] = df['Database Affiliation'].fillna('Unknown')
df['Publication Institutions'] = df['Publication Institutions'].fillna('Unknown')
df['Topic'] = df['Topic'].fillna('Unknown')

In [7]:
# 2. Clean the Source column
# Extract only the URL if it exists, otherwise replace with "Unknown"
def extract_link(source):
    match = re.search(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(source))
    return match.group(0) if match else 'Unknown'

df['Source'] = df['Source'].apply(extract_link)

print(df.head())

NameError: name 're' is not defined

In [8]:
# Remove the "Review Flag" column
if 'Review Flag' in df.columns:
    df = df.drop(columns=['Review Flag'])

In [9]:
# Remove duplicates
df = df.drop_duplicates()

In [10]:
# 5. Save the cleaned dataset
output_path = 'Data_clean/ViCom_Publications_Cleaned.csv'
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to {output_path}")

Cleaned dataset saved to Data_clean/ViCom_Publications_Cleaned.csv


# Change the file to Resercher and Text (topics + abstracts + titles)

In [ ]:
import pandas as pd

# Load the main publication file
publication_file_path = "Data_clean/03.ViCom_Publications_OpenAlex_Cleaned.csv"
publications = pd.read_csv(publication_file_path)

# Column names
researcher_column = 'Researcher'
title_column = 'Title'
abstract_column = 'Abstract'
topic_column = 'Topic'

# Combine Title, Abstract, and Topic into one "Text" field
def combine_text(group):
    combined_text = " ".join(
        filter(None, group[title_column].astype(str) + " " +
               group[abstract_column].astype(str) + " " +
               group[topic_column].fillna("").astype(str))
    )
    return combined_text.strip() if combined_text.strip() else "Unknown"

# Group by researcher and aggregate their data
aggregated_publications = publications.groupby(researcher_column).apply(
    lambda x: pd.Series({
        'Full Name': x[researcher_column].iloc[0],
        'Text': combine_text(x)
    })
).reset_index(drop=True)

# Ensure all researchers in the file are included
all_researchers = publications[[researcher_column]].drop_duplicates().rename(columns={researcher_column: 'Full Name'})
aggregated_publications = all_researchers.merge(
    aggregated_publications, how='left', left_on='Full Name', right_on='Full Name'
)
aggregated_publications['Text'] = aggregated_publications['Text'].fillna("Unknown")

# Save the processed data
output_file_path = "Data_clean/06. Processed_Researcher_Data.csv"
aggregated_publications.to_csv(output_file_path, index=False)

print(f"Processed data saved to {output_file_path}")

Processed data saved to Data_clean/06. Processed_Researcher_Data.csv


/var/folders/fy/9m_z60cj71x0z7l1gf4xtfjh0000gn/T/ipykernel_29190/555101752.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_publications = publications.groupby(researcher_column).apply(


# Adding Bio data from webstite to text

In [16]:
import pandas as pd

# Update file paths
processed_researcher_file = "Data_clean/06. Processed_Researcher_Data.csv"
participants_geo_file = "Data_clean/01_participants_with_geo.csv"

# Load the datasets
processed_researchers = pd.read_csv(processed_researcher_file)
participants_geo = pd.read_csv(participants_geo_file)

# Merge the two datasets on the full name
merged_data = processed_researchers.merge(
    participants_geo[['full_name', 'Bio']],
    left_on='Full Name',
    right_on='full_name',
    how='left'
)

# Combine the `Text` and `Bio` fields into a single `Text` column
merged_data['Text'] = merged_data['Text'].fillna('') + ' ' + merged_data['Bio'].fillna('')

# Drop the unnecessary `full_name` and `Bio` columns
merged_data.drop(columns=['full_name', 'Bio'], inplace=True)

# Save the updated dataset to a new CSV file
output_file_path = "Data_clean/06. Processed_Researcher_Data.csv"
merged_data.to_csv(output_file_path, index=False)

print(f"Updated data with Bio added to Text saved to {output_file_path}.")


Updated data with Bio added to Text saved to Data_clean/06. Processed_Researcher_Data.csv.
